<a href="https://colab.research.google.com/github/vishnuy/helloworld/blob/master/searchsimilar_audio_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchaudio transformers faiss-cpu librosa tqdm

In [ ]:
import os
import requests
from tqdm import tqdm

In [ ]:
AUDIO_URLS = [
#    "https://www2.cs.uic.edu/~i101/SoundFiles/StarWars60.wav",
#    "https://www2.cs.uic.edu/~i101/SoundFiles/BabyElephantWalk60.wav",
#    "https://www2.cs.uic.edu/~i101/SoundFiles/CantinaBand60.wav",
      "/content/audios/lion-roar-sound.mp3",
      "/content/audios/water-flowing-water-noise-and-sound.mp3"
]

#os.makedirs("audios", exist_ok=True)
audio_paths = []

audio_paths.append("/content/audios/lion-roar-sound.mp3")
audio_paths.append("/content/audios/water-flowing-water-noise-and-sound.mp3")
'''
for i, url in enumerate(tqdm(AUDIO_URLS, desc="Downloading audio files")):
    response = requests.get(url)
    if response.status_code == 200:
        path = f"audios/audio_{i}.wav"
        with open(path, "wb") as f:
            f.write(response.content)
        audio_paths.append(path)
'''

print(audio_paths)

['/content/audios/lion-roar-sound.mp3', '/content/audios/water-flowing-water-noise-and-sound.mp3']


In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2Model, Wav2Vec2Processor

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "facebook/wav2vec2-base-960h"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import librosa
import numpy as np

def get_audio_embedding(path):
    waveform, sr = librosa.load(path, sr=16000)
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state  # shape: (batch, time, features)
        embedding = hidden_states.mean(dim=1).squeeze().cpu().numpy()  # mean pooling
    return embedding


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.9 MB/s eta 0:00:00


In [ ]:
import faiss

embeddings = []
for path in tqdm(audio_paths, desc="Embedding audios"):
    vec = get_audio_embedding(path)
    embeddings.append(vec)

embeddings = np.stack(embeddings).astype("float32")

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

faiss.write_index(index, "audio_index.faiss")
with open("audio_paths.txt", "w") as f:
    f.writelines([p + "\n" for p in audio_paths])

Embedding audios: 100%|██████████| 2/2 [00:25<00:00, 12.73s/it]


In [ ]:
def search_similar_audio(query_path, top_k=3):
    index = faiss.read_index("audio_index.faiss")
    with open("audio_paths.txt", "r") as f:
        paths = [line.strip() for line in f.readlines()]

    query_vec = get_audio_embedding(query_path).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_vec, top_k)

    print(f"Query audio: {query_path}")
    print("Top matches:")
    for idx, dist in zip(indices[0], distances[0]):
        print(f"{paths[idx]} - Distance: {dist}")


In [ ]:
search_similar_audio("audios/tiger-light-roar.mp3", top_k=2)


Query audio: audios/tiger-light-roar.mp3
Top matches:
/content/audios/lion-roar-sound.mp3 - Distance: 3.3087899684906006
/content/audios/water-flowing-water-noise-and-sound.mp3 - Distance: 20.596378326416016
